<a href="https://colab.research.google.com/github/SaumyaGupta-99/sarcasm_detection/blob/master/sarcasm_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive
%cd My Drive
%ls

/content/drive
/content/drive/My Drive
'Colab Notebooks'/              idd20k_lite.zip     sarcasm_detection/
'cvupdated (1).pdf'             iddx/              'SAUMYA GUPTA.jpg'
 cvupdated.pdf                 'Internship .gdoc'   saumya.pdf
 Digital_Image_Processing.pdf  'PHP PDO.gslides'    textgeneration/
 emotion_detection/            'PHP PDO.pptx'       toxic-comment/
 idd/                          'Previous QP'/


In [0]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import seaborn as sns
import nltk
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
# import json
# with open('/content/drive/My Drive/sarcasm_detection/Sarcasm_Headlines_Dataset_v2.json') as f:
#    data1 = json.load(f)
# print (data1)
# with open('/content/drive/My Drive/sarcasm_detection/Sarcasm_Headlines_Dataset.json') as f:
#    data2 = json.load(f)
# print (data2)
train1=pd.read_json('/content/drive/My Drive/sarcasm_detection/Sarcasm_Headlines_Dataset_v2.json',lines=True)
train2=pd.read_json("/content/drive/My Drive/sarcasm_detection/Sarcasm_Headlines_Dataset.json",lines=True)

In [4]:
train1.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [5]:
data=train1.append(train2,ignore_index=True)
data.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,article_link,headline,is_sarcastic
0,https://www.theonion.com/thirtysomething-scien...,thirtysomething scientists unveil doomsday clo...,1
1,https://www.huffingtonpost.com/entry/donna-edw...,dem rep. totally nails why congress is falling...,0
2,https://www.huffingtonpost.com/entry/eat-your-...,eat your veggies: 9 deliciously different recipes,0
3,https://local.theonion.com/inclement-weather-p...,inclement weather prevents liar from getting t...,1
4,https://www.theonion.com/mother-comes-pretty-c...,mother comes pretty close to using word 'strea...,1


In [6]:
data['headline']=data['headline'].str.replace("[^a-zA-Z]",' ')
data['headline'].head()
hl_cleansed = []

0    thirtysomething scientists unveil doomsday clo...
1    dem rep  totally nails why congress is falling...
2    eat your veggies    deliciously different recipes
3    inclement weather prevents liar from getting t...
4    mother comes pretty close to using word  strea...
Name: headline, dtype: object

In [7]:
data['headline']=data['headline'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
data['headline'].head()

0    thirtysomething scientists unveil doomsday clo...
1    dem rep totally nails why congress falling sho...
2       eat your veggies deliciously different recipes
3    inclement weather prevents liar from getting work
4    mother comes pretty close using word streaming...
Name: headline, dtype: object

In [8]:
tokenized_words=data['headline'].apply(lambda x: x.split())
tokenized_words.head()

0    [thirtysomething, scientists, unveil, doomsday...
1    [dem, rep, totally, nails, why, congress, fall...
2    [eat, your, veggies, deliciously, different, r...
3    [inclement, weather, prevents, liar, from, get...
4    [mother, comes, pretty, close, using, word, st...
Name: headline, dtype: object

In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [10]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()   
tokenized_words=tokenized_words.apply(lambda x: [lemmatizer.lemmatize(i) for i in x])
tokenized_words.head()

0    [thirtysomething, scientist, unveil, doomsday,...
1    [dem, rep, totally, nail, why, congress, falli...
2    [eat, your, veggie, deliciously, different, re...
3    [inclement, weather, prevents, liar, from, get...
4    [mother, come, pretty, close, using, word, str...
Name: headline, dtype: object

In [11]:
for i in range(len(tokenized_words)):
  tokenized_words[i]=" ".join(tokenized_words[i])
data['headline']=tokenized_words
data['headline'].head()

0    thirtysomething scientist unveil doomsday cloc...
1    dem rep totally nail why congress falling shor...
2         eat your veggie deliciously different recipe
3    inclement weather prevents liar from getting work
4    mother come pretty close using word streaming ...
Name: headline, dtype: object

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer=CountVectorizer(max_df=0.80,min_df=2,stop_words='english')
bow=bow_vectorizer.fit_transform(data['headline'])

In [13]:
n=(int)(len(data)*0.9)
print(n)

49795


In [37]:
#Decisiontree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
train_bow = bow[:n]
test_bow = bow[n:]
y=data['is_sarcastic'][:n]
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow,y,random_state=42, test_size=0.3)
clf = DecisionTreeClassifier()
clf = clf.fit(xtrain_bow,ytrain)
prediction=clf.predict_proba(xvalid_bow)
prediction_int = prediction[:,1] >= 0.3
prediction_int=prediction_int.astype(np.int)
f1_score(yvalid, prediction_int)

0.8693663789924025

In [28]:
test_pred = clf.predict_proba(test_bow)
test_pred_int = test_pred[:,1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)
test=data[n:]
test['is_sarcastic'] = test_pred_int
test.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,article_link,headline,is_sarcastic
49795,https://www.huffingtonpost.com/entry/how-to-tr...,how transform your relationship with money pla...,0
49796,https://www.theonion.com/george-kennedys-honor...,george kennedy honor riding internal breath fr...,1
49797,https://local.theonion.com/women-s-strike-a-so...,woman strike sobering reality check for subway...,1
49798,https://www.theonion.com/video-game-boss-think...,video game bos thinking should get big glowing...,1
49799,https://local.theonion.com/man-worried-any-cra...,man worried any crazy person could get hand co...,1


In [29]:
#Random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
train_bow = bow[:n]
test_bow = bow[n:]
y=data['is_sarcastic'][:n]
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow,y,random_state=42, test_size=0.3)
clfrf=RandomForestClassifier(n_estimators=100)
clfrf = clfrf.fit(xtrain_bow,ytrain)
prediction=clf.predict_proba(xvalid_bow)
prediction_int = predictiona(xvalid_b.5
prediction_int=prediction_ia(xvalid_bnp.int)
f1_score(yvalid, predictiona(xvalid_b

0.8707844439524759

In [30]:
test_pred = clfrf.predict_proba(test_bow)
test_pred_int = test_pred[:,1] >= 0.5
test_pred_int = test_pred_int.astype(np.int)
test=data[n:]
test['is_sarcastic'] = test_pred_int
test.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,article_link,headline,is_sarcastic
49795,https://www.huffingtonpost.com/entry/how-to-tr...,how transform your relationship with money pla...,0
49796,https://www.theonion.com/george-kennedys-honor...,george kennedy honor riding internal breath fr...,1
49797,https://local.theonion.com/women-s-strike-a-so...,woman strike sobering reality check for subway...,1
49798,https://www.theonion.com/video-game-boss-think...,video game bos thinking should get big glowing...,0
49799,https://local.theonion.com/man-worried-any-cra...,man worried any crazy person could get hand co...,1


In [51]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
import numpy as np
max_features=2000
max_token = len(max(data['headline']))
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data['headline'])
sequences = tokenizer.texts_to_sequences(data['headline'])
X = pad_sequences(sequences, maxlen=max_token)
from sklearn.model_selection import train_test_split
Y = data['is_sarcastic'].values
Y = np.vstack(Y)
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3, random_state = 42)
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
embed_dim = 64
model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length =(max_token)))
model.add(LSTM(96, dropout=0.2, recurrent_dropout=0.2, activation='relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 57, 64)            128000    
_________________________________________________________________
lstm_5 (LSTM)                (None, 96)                61824     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 97        
_________________________________________________________________
activation_5 (Activation)    (None, 1)                 0         
Total params: 189,921
Trainable params: 189,921
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
train_bow.shape

(49795, 21490)

In [53]:
epoch = 10
batch_size = 128
model.fit(X_train, Y_train, epochs = epoch, batch_size=batch_size, verbose = 2)




Epoch 1/10





 - 43s - loss: 0.4797 - acc: 0.7578
Epoch 2/10
 - 41s - loss: 0.3522 - acc: 0.8428
Epoch 3/10
 - 41s - loss: 0.3279 - acc: 0.8576
Epoch 4/10
 - 41s - loss: 0.3052 - acc: 0.8677
Epoch 5/10
 - 41s - loss: 0.2872 - acc: 0.8752
Epoch 6/10
 - 42s - loss: 0.2695 - acc: 0.8828
Epoch 7/10
 - 42s - loss: 0.2535 - acc: 0.8895
Epoch 8/10
 - 42s - loss: 0.2393 - acc: 0.8955
Epoch 9/10
 - 42s - loss: 0.2242 - acc: 0.9012
Epoch 10/10
 - 42s - loss: 0.2128 - acc: 0.9061


In [59]:
result = model.predict(X_test)
crr=0
for i in range(len(X_test)):
  if(np.around(result[i])==np.around(Y_test[i])):
    crr=crr+1
acc=crr/(len(X_test))
print(acc)

0.861678414362311
